In [1]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -U \
langchain-community==0.3.5 \
pymupdf==1.24.13 \
faiss-gpu==1.7.2 \
SQLAlchemy==2.0.35 \
dataclasses-json==0.6.7 \
httpx-sse==0.4.0 \
langchain-core==0.3.15 \
langchain==0.3.7 \
pydantic-settings==2.6.1 \
marshmallow==3.23.1 \
python-dotenv==1.0.1 \
typing-inspect==0.9.0 \
mypy-extensions==1.0.0

INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 28.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langsmith
    Fo

In [4]:
import torch
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HypotheticalDocumentEmbedder
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.prompts import PromptTemplate


# PDF 로드 및 텍스트 분할
loader = PyMuPDFLoader("/content/drive/MyDrive/Colab Notebooks/pdf/BBS_202402151054353090.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
text_contents = [doc.page_content for doc in texts]

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 벡터 스토어 설정
vectorstore = FAISS.from_texts(text_contents, embeddings)

# Qwen/Qwen2.5-1.5B-Instruct 모델 설정
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,  # 생성할 최대 새 토큰 수
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        device=0 if torch.cuda.is_available() else -1
    )
)

# 사용자 정의 프롬프트 템플릿 생성
custom_prompt = PromptTemplate(
    input_variables=["question"],
    template="다음 질문에 대한 가상의 문서를 생성해주세요: {question}\n\n문서:"
)

# HyDE 설정
hyde = HypotheticalDocumentEmbedder.from_llm(
    llm=llm,
    base_embeddings=embeddings,
    custom_prompt=custom_prompt
)

# 쿼리 실행
query = "국내 고령화 전망에 대해 알려주세요"
hyde_embedding = hyde.embed_query(query)

# vectorstore를 사용하여 검색
results = vectorstore.similarity_search_by_vector(hyde_embedding)

# hyde.embed_query() 호출 전후에 중간 결과 출력
print("생성된 가상 문서:")
print(llm(custom_prompt.format(question=query)))
print("생성된 임베딩:", hyde_embedding[:10])  # 임베딩의 처음 10개 값만 출력

# 결과 출력
for i, doc in enumerate(results[:5]):  # 상위 5개 결과만 출력
    print(f"관련 문서 {i+1}:")
    print(doc.page_content[:100] + "...\n")

<ipython-input-4-e88636a8bf8f>:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-4-e88636a8bf8f>:29: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(


생성된 가상 문서:


<ipython-input-4-e88636a8bf8f>:64: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm(custom_prompt.format(question=query)))


다음 질문에 대한 가상의 문서를 생성해주세요: 국내 고령화 전망에 대해 알려주세요

문서: 국내 고령화 전망

국내 고령화는 지속적인 현대화와 함께 발생하는 사회적 현상을 의미합니다. 이전과 달리, 고령화가 빠르게 진행되고 있으며, 그 결과로 인한 변화들은 우리 사회에 영향을 미치고 있습니다.

이러한 고령화 현상은 주로 인구 성장과 기술 발달, 경제 변동 등 여러 요인으로부터 비롯됩니다. 인구는 점차적으로 줄어들면서 고령화가 이루어지지만, 동시에 산업과 서비스 분야에서의 기술 혁신은 새로운 일자리를 창출하고 있습니다.

그러나 고령화는 다양한 사회 문제를 초래할 수 있습니다. 특히 노인들의 생활 질 하락, 장애인 지원 부족, 교육 및 재취업 문제 등은 많은 사회적 불평등을 가져올 가능성이 있습니다. 또한, 이에 따른 인프라 구축과 복지 정책 확립도 곤란하게 됩니다.

따라서, 이러한 문제들을 해결하기 위한 대안은 다양합니다. 먼저, 긴급한 인프라
생성된 임베딩: [-0.14133146405220032, 0.22328588366508484, 0.16015151143074036, -0.12375763058662415, 0.1869443953037262, 0.02357862889766693, -0.22758175432682037, 0.1414964497089386, -0.05011116713285446, 0.16265949606895447]
관련 문서 1:
고령화시대 해결을 위한 기술개발
- 3 -
Ⅰ. 고령화시대의 도래
1
국내외 고령화 현황 및 전망
 고령화(高齡化)란 평균 수명의 증가에 따라 총 인구 중 차지하는 노인의 인구비...

관련 문서 2:
고령화시대 해결을 위한 기술개발
- 5 -
2
고령친화 기술의 필요성
 국내 생산가능인구 감소 및 노년부양비 증가 등 고령화 시대 도래에 따른 문제점을 극복하기 
위한 해결책이 ...

관련 문서 3:
고령화시대 해결을 위한 기술개발
- 7 -
 정부는 고령친화 우수사업자·고령친화 우수제품 지정제도 도입으로